In [3]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from concurrent.futures import ThreadPoolExecutor
from geopy.geocoders import Photon
import folium
import geopandas as gpd
from shapely.geometry import Point
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
dataframe_venda = pd.read_parquet('dados_webscraping/dataframe_imoveis_venda_pronto_para_tratamento.parquet')
display(dataframe_venda.head(2))
dataframe_venda_tratado = dataframe_venda.drop_duplicates(subset='url', keep='first')
dataframe_venda_tratado = dataframe_venda_tratado[dataframe_venda_tratado["finalidade"] == "residencial"]


print("Quantidade de imóveis: ", dataframe_venda_tratado.shape[0])
print("Porcentagem de informações faltantes: ")
display(round(dataframe_venda_tratado.isna().sum()/dataframe_venda_tratado.shape[0] * 100, 2).astype(str) + " %")

,url,endereco,preco,area,andar,quartos,suites,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,condominio,elevador,jardim,quadra_esportiva,academia,finalidade,tipo
0,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Luiz Carlos Sarolli, 2021 - Recreio Dos Ba...",510000.0,61.0,2.0,2.0,1.0,1,900.0,1200.0,False,True,True,True,True,True,True,residencial,apartamento
1,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Dias da Rocha - Copacabana, Rio de Janeiro...",895000.0,100.0,2.0,3.0,NaN,1,2154.0,NaN,False,False,False,False,False,False,False,residencial,apartamento


Quantidade de imóveis:  5838
Porcentagem de informações faltantes: 


url                   0.0 %
endereco             1.04 %
preco                4.69 %
area                  5.4 %
andar               57.26 %
quartos              6.37 %
suites               41.5 %
vagas_de_carro        0.0 %
valor_condominio    21.91 %
iptu                25.81 %
mobiliado             0.0 %
piscina               0.0 %
condominio            0.0 %
elevador              0.0 %
jardim                0.0 %
quadra_esportiva      0.0 %
academia              0.0 %
finalidade            0.0 %
tipo                  0.0 %
dtype: object

In [7]:
dataframe_venda_tratado = dataframe_venda_tratado.drop(columns=['andar', "suites"])
dataframe_venda_tratado = dataframe_venda_tratado.dropna()

print("Quantidade de imóveis: ", dataframe_venda_tratado.shape[0])

Quantidade de imóveis:  3861


In [8]:
# def _geocod_propertys_dataframe(dataframe, user_agent):

#     def _geocode_address(address, geolocator):
#         try:
#             geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)
#             location = geocode(address)
#             return location.latitude if location else None, location.longitude if location else None
#         except:
#             return None, None

#     geolocator = Nominatim(user_agent=user_agent, timeout=10)
#     addresses = dataframe['endereco'].tolist()

#     with ThreadPoolExecutor() as executor:
#         results = list(tqdm(executor.map(lambda addr: _geocode_address(addr, geolocator), addresses), total=len(addresses)))

#     dataframe['latitude'], dataframe['longitude'] = zip(*results)
#     return dataframe

# dataframe_aluguel_tratado = _geocod_propertys_dataframe(dataframe_aluguel_tratado, "Geopy Library")

# display(dataframe_aluguel_tratado.head(2))

dataframe_venda_ref = pd.read_parquet('dados_webscraping/dataframe_imoveis_venda_tratados_com_localizacao_bruto.parquet')[["url", "localizacao","latitude", "longitude"]].drop_duplicates(subset="url", keep="first")
dataframe_venda_tratado = dataframe_venda_tratado.merge(dataframe_venda_ref, on="url", how="left")
dataframe_final = dataframe_venda_tratado.dropna()

print(dataframe_final.shape)
display(dataframe_venda_tratado.head(2))

(3861, 20)


,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,condominio,elevador,jardim,quadra_esportiva,academia,finalidade,tipo,localizacao,latitude,longitude
0,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Luiz Carlos Sarolli, 2021 - Recreio Dos Ba...",510000.0,61.0,2.0,1,900.0,1200.0,False,True,True,True,True,True,True,residencial,apartamento,"Rua Luiz Carlos Sarolli, Recreio dos Bandeiran...",-23.02232491986091,-43.509980719670914
1,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Alberto de Faria - Leblon, Rio de Janeiro ...",5800000.0,307.0,4.0,3,2700.0,1300.0,False,False,False,False,False,False,False,residencial,apartamento,None,nan,nan


In [9]:
geometry = [Point(xy) for xy in zip(dataframe_final['longitude'], dataframe_final['latitude'])]
dataframe_final = gpd.GeoDataFrame(dataframe_final, geometry=geometry)
dataframe_final = dataframe_final.set_crs("EPSG:4326") 

print(dataframe_final.shape)
dataframe_final.head(2)

(3861, 21)


,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,elevador,jardim,quadra_esportiva,academia,finalidade,tipo,localizacao,latitude,longitude,geometry
0,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Luiz Carlos Sarolli, 2021 - Recreio Dos Ba...",510000.0,61.0,2.0,1,900.0,1200.0,False,True,...,True,True,True,True,residencial,apartamento,"Rua Luiz Carlos Sarolli, Recreio dos Bandeiran...",-23.02232491986091,-43.509980719670914,POINT (-43.50998 -23.02232)
1,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Alberto de Faria - Leblon, Rio de Janeiro ...",5800000.0,307.0,4.0,3,2700.0,1300.0,False,False,...,False,False,False,False,residencial,apartamento,None,nan,nan,POINT EMPTY


In [10]:
#Associando os imóveis aos bairros

limites_bairros = gpd.read_file('dados_demograficos/Limite_de_Bairros/Limite_de_Bairros.shp')
limites_bairros = limites_bairros.to_crs(epsg=4326)

dataframe_final = gpd.sjoin(dataframe_final, limites_bairros[["nome", "geometry"]], how="left", predicate="within").drop(columns=['index_right'])

In [11]:
def get_properties_near_especific_points(dataframe, dataframe_ref, nome_coluna_referencia):

    dataframe = dataframe.to_crs(epsg=32633)
    dataframe_ref = dataframe_ref.to_crs(epsg=32633)

    dataframe[nome_coluna_referencia] = dataframe["geometry"].apply(
        lambda ponto: dataframe_ref["geometry"].distance(ponto).min())
    dataframe = dataframe.to_crs(epsg=4326)
    
    return dataframe

paradas_de_metro = gpd.read_file('dados_demograficos/Estações_Metrô/Estações_Metrô.shp')

dataframe_final = get_properties_near_especific_points(dataframe_final, paradas_de_metro, "distancia_metro")
dataframe_final.head(2)

,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,quadra_esportiva,academia,finalidade,tipo,localizacao,latitude,longitude,geometry,nome,distancia_metro
0,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Luiz Carlos Sarolli, 2021 - Recreio Dos Ba...",510000.0,61.0,2.0,1,900.0,1200.0,False,True,...,True,True,residencial,apartamento,"Rua Luiz Carlos Sarolli, Recreio dos Bandeiran...",-23.02232491986091,-43.509980719670914,POINT (-43.50998 -23.02232),Recreio dos Bandeirantes,33036.657795
1,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Alberto de Faria - Leblon, Rio de Janeiro ...",5800000.0,307.0,4.0,3,2700.0,1300.0,False,False,...,False,False,residencial,apartamento,None,nan,nan,POINT EMPTY,NaN,NaN


In [12]:
escolas_municipais = gpd.read_file('dados_demograficos/Escolas_Municipais/Escolas_Municipais.shp')

dataframe_final = get_properties_near_especific_points(dataframe_final, escolas_municipais, "distancia_escola")
dataframe_final.head(2)

,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,academia,finalidade,tipo,localizacao,latitude,longitude,geometry,nome,distancia_metro,distancia_escola
0,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Luiz Carlos Sarolli, 2021 - Recreio Dos Ba...",510000.0,61.0,2.0,1,900.0,1200.0,False,True,...,True,residencial,apartamento,"Rua Luiz Carlos Sarolli, Recreio dos Bandeiran...",-23.02232491986091,-43.509980719670914,POINT (-43.50998 -23.02232),Recreio dos Bandeirantes,33036.657795,2029.586333
1,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Alberto de Faria - Leblon, Rio de Janeiro ...",5800000.0,307.0,4.0,3,2700.0,1300.0,False,False,...,False,residencial,apartamento,None,nan,nan,POINT EMPTY,NaN,NaN,NaN


In [13]:
unidade_saude = gpd.read_file('dados_demograficos/Unidades_de_Saúde_Municipais/Unidades_de_Saúde_Municipais.shp')

dataframe_final = get_properties_near_especific_points(dataframe_final, unidade_saude, "distancia_unidade_saude")
dataframe_final.head(2)

,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,finalidade,tipo,localizacao,latitude,longitude,geometry,nome,distancia_metro,distancia_escola,distancia_unidade_saude
0,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Luiz Carlos Sarolli, 2021 - Recreio Dos Ba...",510000.0,61.0,2.0,1,900.0,1200.0,False,True,...,residencial,apartamento,"Rua Luiz Carlos Sarolli, Recreio dos Bandeiran...",-23.02232491986091,-43.509980719670914,POINT (-43.50998 -23.02232),Recreio dos Bandeirantes,33036.657795,2029.586333,4563.542718
1,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Alberto de Faria - Leblon, Rio de Janeiro ...",5800000.0,307.0,4.0,3,2700.0,1300.0,False,False,...,residencial,apartamento,None,nan,nan,POINT EMPTY,NaN,NaN,NaN,NaN


In [14]:
indice_de_desenvolvimento = gpd.read_file('dados_demograficos/Índice_de_Desenvolvimento/Índice_de_Desenvolvimento.shp')
indice_de_desenvolvimento = indice_de_desenvolvimento.to_crs(epsg=4326)

dataframe_final = gpd.sjoin(dataframe_final, indice_de_desenvolvimento[["indic_rend", "indic_lixo", "indic_esgo", "indic_agua","geometry"]], how="left", predicate="within").drop(columns=['index_right'])
dataframe_final.head(2)

,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,...,longitude,geometry,nome,distancia_metro,distancia_escola,distancia_unidade_saude,indic_rend,indic_lixo,indic_esgo,indic_agua
0,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Luiz Carlos Sarolli, 2021 - Recreio Dos Ba...",510000.0,61.0,2.0,1,900.0,1200.0,False,True,...,-43.509980719670914,POINT (-43.50998 -23.02232),Recreio dos Bandeirantes,33036.657795,2029.586333,4563.542718,9.258038,99.710575,77.876606,94.373853
1,https://www.zapimoveis.com.br/imovel/venda-apa...,"Rua Alberto de Faria - Leblon, Rio de Janeiro ...",5800000.0,307.0,4.0,3,2700.0,1300.0,False,False,...,nan,POINT EMPTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
dataframe_final.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3861 entries, 0 to 3860
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   url                      3861 non-null   object  
 1   endereco                 3861 non-null   object  
 2   preco                    3861 non-null   float64 
 3   area                     3861 non-null   float64 
 4   quartos                  3861 non-null   float64 
 5   vagas_de_carro           3861 non-null   int64   
 6   valor_condominio         3861 non-null   float64 
 7   iptu                     3861 non-null   float64 
 8   mobiliado                3861 non-null   bool    
 9   piscina                  3861 non-null   bool    
 10  condominio               3861 non-null   bool    
 11  elevador                 3861 non-null   bool    
 12  jardim                   3861 non-null   bool    
 13  quadra_esportiva         3861 non-null   bool    
 14  acade

In [16]:
dataframe_final.to_parquet('dados_webscraping/dataframe_imoveis_venda_pronto_para_analise.parquet')
